In [1]:
!pip install vectorbt
!pip install condacolab
import condacolab
condacolab.install()
!conda install -c conda-forge ta-lib
!pip install vectorbt
!pip install pandas_ta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.6/34.6 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 40.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.41.1
    Uninstalling llvmlite-0.41.1:
      Successfully uninstalled llvmlite-0.41.1
  Attempting uninstall: numba
    Found existing installation: numba 0.58.1
    Uninstalling numba-0.58.1:
      Success

⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.11.0-0/Mambaforge-23.11.0-0-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:12
🔁 Restarting kernel...
Channels:
 - conda-forge
Platform: linux-64
Solving environment: - \ | done


==> WARNING: A newer version of conda exists. <==
    current version: 23.11.0
    latest version: 24.5.0

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - ta-lib


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2024.6.2   |       hbcca054_0         152 KB  conda-forge
    certifi-2024.6.2           |     pyhd8ed1ab_0         157 KB  conda-forge
    libblas-3.9.0              |22_linux64_openblas          14 KB  conda-forge
    libcblas-3.

In [1]:
import vectorbt as vbt
import talib as ta
import requests
import json
import math
import pandas_datareader as web
import numpy as np
import pandas as pd
import time
import datetime
import tensorflow as tf
import pandas_ta as taa
import matplotlib.pyplot as plt


In [2]:
class GetBinanceData:
    @staticmethod
    def get_data(symbol, interval, startDate=datetime.datetime(2020, 1, 1), endDate=datetime.datetime.now()):
        dataList = []
        current_date = datetime.datetime.now()
        start_date = startDate

        delta_units = GetBinanceData.convert_to_minutes(interval)
        print(delta_units)
        delta = datetime.timedelta(minutes=delta_units)

        while start_date < current_date:
            end_date = start_date + delta
            if end_date > current_date:
                end_date = current_date
            # Convert datetime objects to milliseconds
            start_ms = int(start_date.timestamp() * 1000)
            end_ms = int(end_date.timestamp() * 1000)

            start_str = start_date.strftime('%Y-%m-%d %H:%M:%S')
            end_str = end_date.strftime('%Y-%m-%d %H:%M:%S')

            print(f"Fetching data from {start_str} to {end_str}")
            data = GetBinanceData.get_historical_data(symbol, interval, start_ms, end_ms)
            if data:
                dataList.extend(data)
            start_date = end_date
            time.sleep(1)

        return dataList

    @staticmethod
    def get_historical_data(symbol, interval, start_time, end_time):
        url = f"https://api.binance.us/api/v3/klines"
        params = {
            'symbol': symbol.upper(),
            'interval': interval,
            'startTime': start_time,
            'endTime': end_time,
            'limit': 1000
        }
        print(params)
        response = requests.get(url, params=params)
        if response.status_code == 200:
            data = response.json()
            return data
        else:

            print(f"Failed to fetch data. Status code: {response.json()}")
            return None

    @staticmethod
    def convert_to_minutes(time_str):
        time_mapping = {
            'm': 1,
            'h': 60,
            'd': 1440,
            'w': 10080,
            'M': 43200
        }

        num = int(time_str[:-1])
        unit = time_str[-1]

        return (num * time_mapping[unit])*1000

    def data_to_dataframe(data):
      # Define the column names based on Binance API response
      columns = [
          "timestamp", "open", "high", "low", "close", "volume",
          "Close Time", "Quote Asset Volume", "Number of Trades",
          "Taker Buy Base Asset Volume", "Taker Buy Quote Asset Volume", "Ignore"
      ]
      # Convert the data to a pandas DataFrame
      df = pd.DataFrame(data, columns=columns)
      # Convert timestamp columns to datetime
      df["timestamp"] = pd.to_datetime(df['timestamp'])
      df["Close Time"] = pd.to_datetime(df["Close Time"], unit='ms')
      # Convert numeric columns to float
      numeric_columns = ["open", "high", "low", "close", "volume", "Quote Asset Volume",
                        "Taker Buy Base Asset Volume", "Taker Buy Quote Asset Volume"]
      df[numeric_columns] = df[numeric_columns].astype(float)
      return df

    @staticmethod
    def get_data_dataFrame(symbol, interval, startDate=datetime.datetime(2020, 1, 1), endDate=datetime.datetime.now()):
        data = GetBinanceData.get_data(symbol, interval, startDate, endDate)
        return GetBinanceData.data_to_dataframe(data)

In [65]:
timeperiod = '1h'

symbol = "solusdt"
data = GetBinanceData.get_data(symbol, timeperiod, startDate=datetime.datetime(2021, 8, 1), endDate=datetime.datetime.now())


60000
Fetching data from 2021-08-01 00:00:00 to 2021-09-11 16:00:00
{'symbol': 'SOLUSDT', 'interval': '1h', 'startTime': 1627776000000, 'endTime': 1631376000000, 'limit': 1000}
Fetching data from 2021-09-11 16:00:00 to 2021-10-23 08:00:00
{'symbol': 'SOLUSDT', 'interval': '1h', 'startTime': 1631376000000, 'endTime': 1634976000000, 'limit': 1000}
Fetching data from 2021-10-23 08:00:00 to 2021-12-04 00:00:00
{'symbol': 'SOLUSDT', 'interval': '1h', 'startTime': 1634976000000, 'endTime': 1638576000000, 'limit': 1000}
Fetching data from 2021-12-04 00:00:00 to 2022-01-14 16:00:00
{'symbol': 'SOLUSDT', 'interval': '1h', 'startTime': 1638576000000, 'endTime': 1642176000000, 'limit': 1000}
Fetching data from 2022-01-14 16:00:00 to 2022-02-25 08:00:00
{'symbol': 'SOLUSDT', 'interval': '1h', 'startTime': 1642176000000, 'endTime': 1645776000000, 'limit': 1000}
Fetching data from 2022-02-25 08:00:00 to 2022-04-08 00:00:00
{'symbol': 'SOLUSDT', 'interval': '1h', 'startTime': 1645776000000, 'endTime'

In [66]:
dataframe = GetBinanceData.data_to_dataframe(data)
dataframe = dataframe.dropna()

dataframe.head()



,timestamp,open,high,low,close,volume,Close Time,Quote Asset Volume,Number of Trades,Taker Buy Base Asset Volume,Taker Buy Quote Asset Volume,Ignore
0,1970-01-01 00:27:07.776000,36.8003,36.8130,35.6575,36.1063,23503.61,2021-08-01 00:59:59.999,845859.011743,1064,12126.08,436274.671111,0
1,1970-01-01 00:27:07.779600,36.1161,36.1469,35.3900,35.6450,13563.89,2021-08-01 01:59:59.999,483881.302693,430,2843.24,101291.422500,0
2,1970-01-01 00:27:07.783200,35.6791,35.7404,35.3021,35.6365,7008.81,2021-08-01 02:59:59.999,248254.949724,232,1634.14,57889.433052,0
3,1970-01-01 00:27:07.786800,35.6597,36.2757,35.5547,36.1108,5108.25,2021-08-01 03:59:59.999,183569.930400,307,1909.98,68650.753902,0
4,1970-01-01 00:27:07.790400,36.0689,36.1524,35.5227,35.5227,2479.35,2021-08-01 04:59:59.999,88716.873535,152,949.13,33936.816941,0


In [67]:
dataframe['ha_close'] = (dataframe[['open', 'high', 'low', 'close']].sum(axis=1)) / 4
dataframe['ha_open'] = (dataframe['open'].shift(1) + dataframe['close'].shift(1)) / 2
dataframe['ha_open'].iloc[0] = dataframe['open'].iloc[0]  # The first open value is just the original open value

dataframe['ha_high'] = dataframe[['high', 'ha_open', 'ha_close']].max(axis=1)
dataframe['ha_low'] = dataframe[['low', 'ha_open', 'ha_close']].min(axis=1)

print(dataframe.head())

                   timestamp     open     high      low    close    volume  \
0 1970-01-01 00:27:07.776000  36.8003  36.8130  35.6575  36.1063  23503.61   
1 1970-01-01 00:27:07.779600  36.1161  36.1469  35.3900  35.6450  13563.89   
2 1970-01-01 00:27:07.783200  35.6791  35.7404  35.3021  35.6365   7008.81   
3 1970-01-01 00:27:07.786800  35.6597  36.2757  35.5547  36.1108   5108.25   
4 1970-01-01 00:27:07.790400  36.0689  36.1524  35.5227  35.5227   2479.35   

               Close Time  Quote Asset Volume  Number of Trades  \
0 2021-08-01 00:59:59.999       845859.011743              1064   
1 2021-08-01 01:59:59.999       483881.302693               430   
2 2021-08-01 02:59:59.999       248254.949724               232   
3 2021-08-01 03:59:59.999       183569.930400               307   
4 2021-08-01 04:59:59.999        88716.873535               152   

   Taker Buy Base Asset Volume  Taker Buy Quote Asset Volume Ignore  \
0                     12126.08                 436274.671

<ipython-input-67-5b9df8d39735>:3: FutureWarning:

ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


<ipython-input-67-5b9df8d39735>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documenta

In [68]:
# Supertrend Parameters
atr_periods = [12, 10, 11]
factors = [3.0, 1.0, 2.0]

for i in range(3):
    # Calculate ATR
    dataframe[f'ATR_{i}'] = ta.ATR(dataframe['ha_high'], dataframe['ha_low'], dataframe['ha_close'], timeperiod=atr_periods[i])

    # Initialize Supertrend columns
    dataframe[f'basic_ub_{i}'] = (dataframe['ha_high'] + dataframe['ha_low']) / 2 + factors[i] * dataframe[f'ATR_{i}']
    dataframe[f'basic_lb_{i}'] = (dataframe['ha_high'] + dataframe['ha_low']) / 2 - factors[i] * dataframe[f'ATR_{i}']
    dataframe[f'final_ub_{i}'] = np.nan
    dataframe[f'final_lb_{i}'] = np.nan
    dataframe[f'supertrend_{i}'] = np.nan
    dataframe[f'direction_{i}'] = np.nan

    # Calculate Supertrend
    for j in range(atr_periods[i], len(dataframe)):
        if dataframe['ha_close'][j-1] <= dataframe[f'final_ub_{i}'][j-1]:
            dataframe[f'final_ub_{i}'][j] = min(dataframe[f'basic_ub_{i}'][j], dataframe[f'final_ub_{i}'][j-1])
        else:
            dataframe[f'final_ub_{i}'][j] = dataframe[f'basic_ub_{i}'][j]

        if dataframe['ha_close'][j-1] >= dataframe[f'final_lb_{i}'][j-1]:
            dataframe[f'final_lb_{i}'][j] = max(dataframe[f'basic_lb_{i}'][j], dataframe[f'final_lb_{i}'][j-1])
        else:
            dataframe[f'final_lb_{i}'][j] = dataframe[f'basic_lb_{i}'][j]

        if dataframe['ha_close'][j] <= dataframe[f'final_ub_{i}'][j]:
            dataframe[f'supertrend_{i}'][j] = dataframe[f'final_ub_{i}'][j]
            dataframe[f'direction_{i}'][j] = -1
        else:
            dataframe[f'supertrend_{i}'][j] = dataframe[f'final_lb_{i}'][j]
            dataframe[f'direction_{i}'][j] = 1

# Calculate RSI
dataframe['rsi'] = ta.RSI(dataframe['ha_high'], timeperiod=14)

# Calculate EMAs
dataframe['ema_9'] = ta.EMA(dataframe['ha_high'], timeperiod=5)
dataframe['ema_200'] = ta.EMA(dataframe['ha_high'], timeperiod=200)

Streaming output truncated to the last 5000 lines.
<ipython-input-68-72947158c732>:20: FutureWarning:

ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


<ipython-input-68-72947158c732>:20: SettingWithCopyWarning:


A value is trying to be set on a copy of a sli

KeyboardInterrupt: 

In [69]:
# Supertrend Parameters
atr_periods = [12, 10, 11]
factors = [3.0, 1.0, 2.0]
superTrend1 = taa.supertrend(dataframe['ha_high'], dataframe['ha_low'], dataframe['ha_close'], length=13, multiplier=2)
superTrend2 = taa.supertrend(dataframe['ha_high'], dataframe['ha_low'], dataframe['ha_close'], length=12, multiplier=3)
superTrend3 = taa.supertrend(dataframe['ha_high'], dataframe['ha_low'], dataframe['ha_close'], length=14, multiplier=4)

# SUPERTd_7_3.0
dataframe['direction_0'] = superTrend1["SUPERTd_13_2.0"]
dataframe['direction_1'] = superTrend2["SUPERTd_12_3.0"]
dataframe['direction_2'] = superTrend3["SUPERTd_14_4.0"]

# Calculate RSI
dataframe['rsi'] = ta.RSI(dataframe['ha_low'], timeperiod=14)

# Calculate EMAs
dataframe['ema_9'] = ta.EMA(dataframe['ha_low'], timeperiod=9)
dataframe['ema_200'] = ta.EMA(dataframe['high'], timeperiod=200)
dataframe.dropna()

,timestamp,open,high,low,close,volume,Close Time,Quote Asset Volume,Number of Trades,Taker Buy Base Asset Volume,...,basic_lb_0,final_ub_0,final_lb_0,supertrend_0,direction_0,direction_1,direction_2,rsi,ema_9,ema_200
199,1970-01-01 00:27:08.492400,37.3198,37.7814,37.1140,37.7814,1005.60,2021-08-09 07:59:59.999,3.760717e+04,182,648.30,...,35.543102,38.962844,35.543102,38.962844,-1,-1,-1,45.177414,37.087397,36.671948
200,1970-01-01 00:27:08.496000,37.7559,38.6198,37.7559,38.5594,5953.88,2021-08-09 08:59:59.999,2.280764e+05,611,2859.19,...,36.059156,38.962844,36.059156,38.962844,-1,-1,-1,51.219596,37.180037,36.691330
201,1970-01-01 00:27:08.499600,38.4974,39.0000,38.4974,38.9712,3951.66,2021-08-09 09:59:59.999,1.533362e+05,410,2031.87,...,36.511031,38.962844,36.511031,38.962844,1,-1,-1,58.129443,37.375560,36.714301
202,1970-01-01 00:27:08.503200,38.9963,39.1923,38.6306,39.0965,4089.33,2021-08-09 10:59:59.999,1.593993e+05,450,2210.53,...,36.875547,38.962844,36.875547,36.875547,1,1,-1,62.577143,37.626568,36.738958
203,1970-01-01 00:27:08.506800,39.1894,39.3251,38.8558,39.2696,2032.22,2021-08-09 11:59:59.999,7.952218e+04,328,861.43,...,37.106880,41.074020,37.106880,41.074020,1,1,-1,64.510299,37.872414,36.764691
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
487,1970-01-01 00:27:09.529200,80.1374,80.1499,78.5935,78.9545,4099.08,2021-08-21 07:59:59.999,3.239865e+05,342,2172.26,...,73.999733,82.440316,74.210371,82.440316,1,1,1,66.889877,78.036603,64.663509
488,1970-01-01 00:27:09.532800,78.8906,79.1175,78.0484,78.5421,4893.15,2021-08-21 08:59:59.999,3.849088e+05,456,2253.33,...,73.498484,82.440316,74.210371,82.440316,1,1,1,62.839981,78.038962,64.807330
489,1970-01-01 00:27:09.536400,78.6428,78.6428,77.5280,77.5453,3524.49,2021-08-21 09:59:59.999,2.747226e+05,469,1410.89,...,72.967954,82.440316,74.210371,82.440316,1,1,1,59.157512,77.936770,64.944996
490,1970-01-01 00:27:09.540000,77.5347,77.8264,76.9921,77.2339,3321.85,2021-08-21 10:59:59.999,2.568356e+05,407,1408.50,...,72.542885,82.440316,74.210371,82.440316,1,1,1,55.547577,77.747836,65.073170


In [70]:
long = (
    (dataframe['direction_0'] == 1) &
    (dataframe['direction_1'] == 1) &
    (dataframe['direction_2'] == 1) &
    (dataframe['rsi'] > 70) &
    (dataframe['ha_high'] > dataframe['ema_9'])
    & (dataframe['ema_200'] < dataframe['ema_9'])
)

short =  (dataframe['ema_200'] > dataframe['ema_9']) | (dataframe['ema_9'] > dataframe['ha_high'])
pf = vbt.Portfolio.from_signals(
    close=dataframe['close'],
    entries=long,
    exits=short,
    init_cash=10000,
    freq='1h',
    sl_stop=0.01,


)

# Performance metrics
print(pf.stats())

# Plot the results
pf.plot().show()


Start                                                 0
End                                               25387
Period                               1057 days 20:00:00
Start Value                                     10000.0
End Value                                  83195.101722
Total Return [%]                             731.951017
Benchmark Return [%]                         253.345538
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                              34.297208
Max Drawdown Duration                 282 days 01:00:00
Total Trades                                        182
Total Closed Trades                                 182
Total Open Trades                                     0
Open Trade PnL                                      0.0
Win Rate [%]                                  29.120879
Best Trade [%]                                72.663854
Worst Trade [%]                               -7

In [ ]:
macd, macdsignal, macdhist = ta.MACD(dataframe["close"])
dataframe['macd'] = macd
dataframe['macdsignal'] = macdsignal
dataframe['macdhist'] = macdhist

# Calculate CCI
dataframe['cci'] = ta.CCI(dataframe["high"], dataframe["low"], dataframe["close"])

In [ ]:
# Define buy and sell signals
buy_signal = (
    (dataframe['macd'] > dataframe['macdsignal']) &
    (dataframe['cci'] <= -48) &
    (dataframe['volume'] > 0)
)

sell_signal = (
    (dataframe['macd'] < dataframe['macdsignal']) &
    (dataframe['cci'] >= 687) &
    (dataframe['volume'] > 0)
)

In [ ]:

pf = vbt.Portfolio.from_signals(
    close=dataframe['close'],
    entries=buy_signal,
    exits=sell_signal,
    init_cash=10000,
    freq='5m',  # 5 minutes
    tp_stop=0.01,
    sl_stop=0.1,  # Stoploss at -20%


)

# Performance metrics
print(pf.stats())

# Plot the results
pf.plot().show()

Start                                                 0
End                                              302311
Period                               1049 days 16:40:00
Start Value                                     10000.0
End Value                                  14683.357426
Total Return [%]                              46.833574
Benchmark Return [%]                          60.281069
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                              59.174617
Max Drawdown Duration                 867 days 10:35:00
Total Trades                                        505
Total Closed Trades                                 504
Total Open Trades                                     1
Open Trade PnL                             -1094.905944
Win Rate [%]                                  90.873016
Best Trade [%]                                 3.776005
Worst Trade [%]                              -17

In [ ]:
# Define the range variables used in valuewhen
rangeUpper = 60
rangeLower = 5
def EWO(dataframe, ema_length=5, ema2_length=35):
    df = dataframe.copy()
    ema1 = ta.EMA(df['close'], timeperiod=ema_length)
    ema2 = ta.EMA(df['close'], timeperiod=ema2_length)
    emadif = (ema1 - ema2) / df['close'] * 100
    return emadif

# Function to calculate valuewhen
# Function to calculate valuewhen
def valuewhen(dataframe, condition, source, occurrence):
    copy = dataframe.copy()
    copy['colFromIndex'] = np.arange(len(copy))
    copy = copy.sort_values(by=[condition, 'colFromIndex'], ascending=False).reset_index(drop=True)
    copy['valuewhen'] = np.where(copy[condition] > 0, copy[source].shift(-occurrence), copy[source])
    copy['barrsince'] = copy['colFromIndex'] - copy['colFromIndex'].shift(-occurrence)
    copy.loc[
        (
            (rangeLower <= copy['barrsince']) &
            (copy['barrsince'] <= rangeUpper)
        )
    , "in_range"] = 1
    copy['in_range'] = copy['in_range'].fillna(0)
    copy = copy.sort_values(by=['colFromIndex'], ascending=True).reset_index(drop=True)
    return copy['valuewhen'], copy['in_range']

# Load historical data
# Assuming 'df' is your DataFrame with historical data and it includes 'timestamp', 'close', 'volume' columns
# Ensure 'timestamp' is the index and is of datetime type

# Add indicators
df['RSI'] = ta.RSI(df['close'], timeperiod=14)

# Correct the call to STOCH
stoch_k, stoch_d = ta.STOCH(df['high'], df['low'], df['close'], fastk_period=10, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
df['slowk'] = stoch_k
df['slowd'] = stoch_d
df['osc'] = df['slowd']

df['EWO'] = EWO(df, 50, 200)
df['ADX'] = ta.ADX(df['high'], df['low'], df['close'], timeperiod=14)

# Calculate pivot lows and highs
df['min'] = df['osc'].rolling(5).min()
df['prevMin'] = np.where(df['min'] > df['min'].shift(), df['min'].shift(), df['min'])
df.loc[
    (
        (df['osc'].shift(1) == df['prevMin'].shift(1)) &
        (df['osc'] != df['prevMin'])
    )
, 'plFound'] = 1

df['max'] = df['osc'].rolling(5).max()
df['prevMax'] = np.where(df['max'] < df['max'].shift(), df['max'].shift(), df['max'])
df.loc[
    (
    (df['osc'].shift(1) == df['prevMax'].shift(1)) &
    (df['osc'] != df['prevMax'])
    )
, 'phFound'] = 1

# Regular Bullish
df['valuewhen_plFound_osc'], df['inrange_plFound_osc'] = valuewhen(df, 'plFound', 'osc', 1)
df.loc[
    (
        (df['osc'] > df['valuewhen_plFound_osc']) &
        (df['inrange_plFound_osc'] == 1)
     )
, 'oscHL'] = 1

df['valuewhen_plFound_low'], df['inrange_plFound_low'] = valuewhen(df, 'plFound', 'low', 1)
df.loc[
    (df['low'] < df['valuewhen_plFound_low'])
    , 'priceLL'] = 1

df.loc[
    (
        (df['priceLL'] == 1) &
        (df['oscHL'] == 1) &
        (df['plFound'] == 1)
    )
    , 'bullCond'] = 1

# Hidden Bullish
df['valuewhen_plFound_osc'], df['inrange_plFound_osc'] = valuewhen(df, 'plFound', 'osc', 1)
df.loc[
    (
        (df['osc'] < df['valuewhen_plFound_osc']) &
        (df['inrange_plFound_osc'] == 1)
     )
, 'oscLL'] = 1

df['valuewhen_plFound_low'], df['inrange_plFound_low'] = valuewhen(df, 'plFound', 'low', 1)
df.loc[
    (df['low'] > df['valuewhen_plFound_low'])
    , 'priceHL'] = 1

df.loc[
    (
        (df['priceHL'] == 1) &
        (df['oscLL'] == 1) &
        (df['plFound'] == 1)
    )
    , 'hiddenBullCond'] = 1

# Regular Bearish
df['valuewhen_phFound_osc'], df['inrange_phFound_osc'] = valuewhen(df, 'phFound', 'osc', 1)
df.loc[
    (
        (df['osc'] < df['valuewhen_phFound_osc']) &
        (df['inrange_phFound_osc'] == 1)
     )
, 'oscLH'] = 1

df['valuewhen_phFound_high'], df['inrange_phFound_high'] = valuewhen(df, 'phFound', 'high', 1)
df.loc[
    (df['high'] > df['valuewhen_phFound_high'])
    , 'priceHH'] = 1

df.loc[
    (
        (df['priceHH'] == 1) &
        (df['oscLH'] == 1) &
        (df['phFound'] == 1)
    )
    , 'bearCond'] = 1

# Hidden Bearish
df['valuewhen_phFound_osc'], df['inrange_phFound_osc'] = valuewhen(df, 'phFound', 'osc', 1)
df.loc[
    (
        (df['osc'] > df['valuewhen_phFound_osc']) &
        (df['inrange_phFound_osc'] == 1)
     )
, 'oscHH'] = 1

df['valuewhen_phFound_high'], df['inrange_phFound_high'] = valuewhen(df, 'phFound', 'high', 1)
df.loc[
    (df['high'] < df['valuewhen_phFound_high'])
    , 'priceLH'] = 1

df.loc[
    (
        (df['priceLH'] == 1) &
        (df['oscHH'] == 1) &
        (df['phFound'] == 1)
    )
    , 'hiddenBearCond'] = 1

# Define buy and sell signals
buy_signal = (
    (df['bullCond'] > 0) &
    (df['valuewhen_plFound_osc'] > 15) &
    (df['valuewhen_plFound_osc'] < 74) &
    (df['RSI'] < 53) &
    (df['RSI'] > 9) &
    (df['ADX'] > 41) &
    (df['ADX'] < 68) &
    (df['slowk'] < 64) &
    (df['slowk'] > 13) &
    (df['volume'] > 0)
)

sell_signal = (
    (df['bearCond'] > 0) &
    (df['volume'] > 0)
)



TypeError: Portfolio.__init__() got an unexpected keyword argument 'trailing_stop_positive'

In [ ]:

pf = vbt.Portfolio.from_signals(
    close=df['close'],
    entries=buy_signal,
    exits=sell_signal,
    init_cash=10000,
    freq='5m',  # 5 minutes
    sl_stop=-0.2,  # Stoploss at -20%
    sl_trail=True,

)

# Performance metrics
print(pf.stats())

# Plot the results
pf.plot().show()

Start                         1970-01-01 00:27:07.776000
End                           1970-01-01 00:28:38.422500
Period                                1048 days 20:10:00
Start Value                                      10000.0
End Value                                        10000.0
Total Return [%]                                     0.0
Benchmark Return [%]                           60.111334
Max Gross Exposure [%]                               0.0
Total Fees Paid                                      0.0
Max Drawdown [%]                                     NaN
Max Drawdown Duration                                NaT
Total Trades                                           0
Total Closed Trades                                    0
Total Open Trades                                      0
Open Trade PnL                                       0.0
Win Rate [%]                                         NaN
Best Trade [%]                                       NaN
Worst Trade [%]                

In [ ]:
rsi_period = 14  # RSI period
df['RSI'] = talib.RSI(df['Close'].values, timeperiod=rsi_period)

# Calculate Bollinger Bands using TA-Lib
bb_period = 20  # Bollinger Bands period
bb_std_dev = 2  # Standard deviation for Bollinger Bands
upper_band, middle_band, lower_band = talib.BBANDS(df['Close'].values, timeperiod=bb_period, nbdevup=bb_std_dev, nbdevdn=bb_std_dev)

# Add Bollinger Bands to DataFrame
df['Upper Band'] = upper_band
df['Middle Band'] = middle_band
df['Lower Band'] = lower_band

In [ ]:
df['SMA20'] = talib.SMA(df['Close'].values, timeperiod=20)
df['SMA200'] = talib.SMA(df['Close'].values, timeperiod=200)

In [ ]:
df.dropna()

df.head()

,Open Time,Open,High,Low,Close,Volume,Close Time,Quote Asset Volume,Number of Trades,Taker Buy Base Asset Volume,...,R2_Daily,S2_Daily,R3_Daily,S3_Daily,RSI,Upper Band,Middle Band,Lower Band,SMA20,SMA200
0,2021-08-01 00:00:00,41454.28,41488.30,41109.41,41358.59,21.140863,2021-08-01 00:04:59.999,872410.880044,433,11.531245,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-08-01 00:05:00,41355.29,41551.34,41355.29,41488.56,4.721500,2021-08-01 00:09:59.999,195809.411916,110,3.437676,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-08-01 00:10:00,41488.56,41508.41,41377.88,41503.84,2.459981,2021-08-01 00:14:59.999,101985.295034,65,1.274901,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-08-01 00:15:00,41487.78,41558.65,41457.12,41535.73,1.549756,2021-08-01 00:19:59.999,64306.439251,28,0.651710,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-08-01 00:20:00,41562.04,41646.60,41556.11,41599.69,2.379079,2021-08-01 00:24:59.999,98961.934668,74,1.197269,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


KeyError: 'Datetime'

In [ ]:
long_condition = (
       ((df['Close'] > df['R1_Daily']) | (df['Close'] > df['R2_Daily']) | (df['Close'] > df['R3_Daily']))




    )

In [ ]:
exit_condition = ()

In [ ]:




portfolio = vbt.Portfolio.from_signals(df['Close'], entries=long_condition,sl_stop=0.03,tp_stop=0.06 , sl_trail=True,freq=timeperiod)




stats = portfolio.stats()
print(stats)
# Print the total return
total_return = portfolio.total_return()
print("Total Return: ", total_return)

Start                                                 0
End                                              301415
Period                               1046 days 14:00:00
Start Value                                       100.0
End Value                                     95.863041
Total Return [%]                              -4.136959
Benchmark Return [%]                          65.370169
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                              49.569796
Max Drawdown Duration                 965 days 23:05:00
Total Trades                                        310
Total Closed Trades                                 310
Total Open Trades                                     0
Open Trade PnL                                      0.0
Win Rate [%]                                  34.193548
Best Trade [%]                                 8.270872
Worst Trade [%]                               -4